In [1]:
import pandas as pd 

In [3]:
import joblib
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn

class StockPredictor:
    def __init__(self):
        self.scaler = MinMaxScaler()
        self.model = None
        self.neural_net = nn.Sequential(
            nn.Linear(15, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )

    def load_model(self, model_path: str, neural_net_path: str):
        self.model = joblib.load(model_path)
        self.neural_net.load_state_dict(torch.load(neural_net_path))
        self.neural_net.eval()

    def fit_scaler(self, X):
        self.scaler.fit(X)

    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        predictions = self.model.predict(X_scaled)
        return predictions

if __name__ == "__main__":
    predictor = StockPredictor()
    predictor.load_model(r"D:\DOT SLASH\RELI_model.pkl", r"D:\DOT SLASH\contrastive_model.pth")

    # Load new data for prediction
    new_data = pd.read_csv(r"D:\DOT SLASH\testing_features.csv")
    new_data.columns = new_data.columns.str.strip()
    new_data["date"] = pd.to_datetime(new_data["date"], errors="coerce")
    new_data.dropna(subset=["date"], inplace=True)
    new_data.set_index("date", inplace=True)
    new_data = new_data.apply(pd.to_numeric, errors="coerce")
    X_new = new_data.values

    # Fit the scaler on the new data
    predictor.fit_scaler(X_new)

    predictions = predictor.predict(X_new)
    print(f"Predictions: {predictions}")

C:\Users\OM\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVR from version 1.6.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Predictions: [ 0.05234946  0.05945187  0.01173665 ... -0.00853084 -0.00148193
 -0.01607401]
